In [1]:
# Useful Imports

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2
import pathlib
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Nadam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model
import csv
from IPython.display import HTML as html_print
from IPython.display import display
from matplotlib.font_manager import FontProperties

In [2]:
if os.path.exists('best_model_attn.h5'):
    os.remove('best_model_attn.h5')

### Downloading the Dakshina dataset

In [3]:
#Downloading
!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
    
#Uncompressing
!tar -xf dakshina_dataset_v1.0.tar

### Pre processing data

In [4]:
def read(data_path, characters = False):
    
    # Returns the (x, y) pair from the dataset
    # If characters == True, the input/output sample would be in the form list of characters, else as string

    with open(data_path, "r", encoding="utf-8") as f:
        lines = [line.split("\t") for line in f.read().split("\n") if line != '']
    
    x, y = [val[1] for val in lines], [val[0] for val in lines]
    '''if characters:
        input, target = [list(inp_str) for inp_str in input], [list(tar_str) for tar_str in target]'''
    return x, y

In [5]:
START_CHAR = '\t'
END_CHAR = '\n'
BLANK_CHAR = ' '
def encode_decode_characters(train_input, train_target, val_input, val_target):
    
    # Returns the encoding for characters to integer (as a dictionary) and decoding for integers to characters (as a list) for input and target data
    # Encoding and decoding of input vocabulary
    
    input_char_enc = {}
    input_char_dec = []
    max_encoder_seq_length = 1
    for string in train_input + val_input:
        max_encoder_seq_length = max(max_encoder_seq_length, len(string))
        for char in string:
            if char not in input_char_enc:
                input_char_enc[char] = len(input_char_dec)
                input_char_dec.append(char)
    if BLANK_CHAR not in input_char_enc:
        input_char_enc[BLANK_CHAR] = len(input_char_dec)
        input_char_dec.append(BLANK_CHAR)
        
    # Encoding and decoding of target vocabulary
    target_char_enc = {}
    target_char_dec = []
    target_char_enc[START_CHAR] = len(target_char_dec)
    target_char_dec.append(START_CHAR)
    max_decoder_seq_length = 1
    for string in train_target + val_target:
        max_decoder_seq_length = max(max_decoder_seq_length, len(string)+2)
        for char in string:
            if char not in target_char_enc:
                target_char_enc[char] = len(target_char_dec)
                target_char_dec.append(char)
    target_char_enc[END_CHAR] = len(target_char_dec)
    target_char_dec.append(END_CHAR)
    if ' ' not in target_char_enc:
        target_char_enc[BLANK_CHAR] = len(target_char_dec)
        target_char_dec.append(BLANK_CHAR)

    print("Number of training samples:", len(train_input))
    print("Number of validation samples:", len(val_input))
    print("Number of unique input tokens:", len(input_char_dec))
    print("Number of unique output tokens:", len(target_char_dec))
    print("Max sequence length for inputs:", max_encoder_seq_length)
    print("Max sequence length for outputs:", max_decoder_seq_length)

    return input_char_enc, input_char_dec, target_char_enc, target_char_dec, max_encoder_seq_length, max_decoder_seq_length

In [6]:
def process_data(input, enc_timesteps, input_char_enc, target = None, dec_timesteps = None, target_char_enc = None):
    
    # Returns the input and target data in a form needed by the Keras embedding layer (i.e) 
    # decoder_input & encoder_input -- (None, timesteps) where each character is encoded by an integer
    # decoder_output -- (None, timesteps, vocabulary size) where the last dimension is the one-hot encoding
    # BLANK_CHAR -- space (equivalent to no meaningful input / blank input)
    
    encoder_input = np.array([[input_char_enc[ch] for ch in string] + [input_char_enc[BLANK_CHAR]] * (enc_timesteps - len(string)) for string in input])

    decoder_input, decoder_target = None, None
    if target is not None and dec_timesteps is not None and target_char_enc is not None:
        
        # START_CHAR -- start of sequence, END_CHAR -- end of sequence
        decoder_input = np.array([[target_char_enc[START_CHAR]] + [target_char_enc[ch] for ch in string] + [target_char_enc[END_CHAR]] 
                                    + [target_char_enc[BLANK_CHAR]] * (dec_timesteps - len(string) - 2) for string in target])
        decoder_target = np.zeros((decoder_input.shape[0], dec_timesteps, len(target_char_enc)), dtype='float32')

        for i in range(decoder_input.shape[0]):
            for t, char_ind in enumerate(decoder_input[i]):
                if t > 0:
                    decoder_target[i,t-1,char_ind] = 1.0
            decoder_target[i,t:,target_char_enc[BLANK_CHAR]] = 1.0

    return encoder_input, decoder_input, decoder_target

In [7]:
train_x, train_y = read('./dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.train.tsv')
val_x, val_y = read('./dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.dev.tsv')
test_x, test_y = read('./dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.test.tsv')

# Assigning encoding and decoding for input and target characters
input_char_enc, input_char_dec, target_char_enc, target_char_dec, max_encoder_seq_length, max_decoder_seq_length = encode_decode_characters(
    train_x, train_y, val_x, val_y)

# Assigning training, validation and test encoder input, decoder input, decoder output
train_enc_x, train_dec_x, train_dec_y = process_data(train_x, max_encoder_seq_length, input_char_enc, train_y, 
                                                                  max_decoder_seq_length, target_char_enc)
val_enc_x, val_dec_x, val_dec_y = process_data(val_x, max_encoder_seq_length, input_char_enc, val_y, 
                                                            max_decoder_seq_length, target_char_enc)
test_enc_x, test_dec_x, test_dec_y = process_data(test_x, max_encoder_seq_length, input_char_enc, test_y, 
                                                               max_decoder_seq_length, target_char_enc)

In [8]:
def convert_to_word(predictions, char_enc, char_dec = None):
    
    # Function to return the predictions after cutting the END_CHAR and BLANK_CHAR s at the end.
    # If char_dec == None, the predictions are in the form of decoded string, otherwise as list of integers
    
    no_samples = len(predictions) if type(predictions) is list else predictions.shape[0]
    pred_words = ['' for _ in range(no_samples)]
    for i, pred_list in enumerate(predictions):
        for l in pred_list:
            # Stop word : END_CHAR
            if l == char_enc[END_CHAR]:
                break
            pred_words[i] += char_dec[l] if char_dec is not None else l
    
    return pred_words

## Seq2Seq Model (with Attention)

### Creating Attention layer
Source referenced : https://github.com/thushv89/attention_keras/blob/master/src/layers/attention.py

In [9]:
from keras.layers import Layer
import keras.backend as K

class AttentionLayer(Layer):
    """
    Source referenced : https://github.com/thushv89/attention_keras/blob/master/src/layers/attention.py
    W_a, U_a, and V_a - 3 sets of weights
    """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            
            """ Step function for computing energy for a single decoder state
            inputs: (batchsize * 1 * de_in_dim)
            states: (batchsize * 1 * de_latent_dim)
            """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch size * en_seq_len * latent_dim
            W_a_dot_s = K.dot(encoder_out_seq, self.W_a)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>', U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)
            if verbose:
                print('Ws+Uh>', Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            # (batch_size, decoder_timesteps, decoder_h_layer_size)
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            # (batch_size, decoder_timesteps, encoder_timesteps)
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

### Building the model

In [10]:
def build_model(encoder_vocab_size, decoder_vocab_size, inp_emb_size=64, n_enc_layers=1, n_dec_layers=1, 
                           h_layer_size=64, cell_type='LSTM', dropout=0, r_dropout=0, cell_activation='tanh'):
    '''
    Function to create a seq2seq model without attention.
    Input :
        encoder_vocab_size -- number of characters in input vocabulary (int)
        decoder_vocab_size -- number of characters in output vocabulary (int)
        inp_emb_size -- size of input embedding layer for encoder and decoder (int, default value : 64)
        n_enc_layers -- number of layers of cell to stack in encoder (int, default value : 1)
        n_dec_layers -- number of layers of cell to stack in decoder (int, default value : 1)
        h_layer_size -- size of hidden layer of the encoder and decoder cells (int, default : 64)
        cell_type -- type of cell used in encoder and decoder (string('LSTM'/ 'GRU'/ 'RNN'), default : 'LSTM')
        dropout -- value of normal dropout (float(between 0 and 1), default : 0.0)
        r_dropout -- value of recurrent dropout (float(between 0 and 1), default : 0.0)
        cell_activation -- type of activation used in the cell (string, default : 'tanh')
    Output :
        model -- (Keras model object)
    '''
    
    # Getting cell type
    get_cell = {
        'RNN': keras.layers.SimpleRNN,
        'GRU': keras.layers.GRU,
        'LSTM': keras.layers.LSTM
    }
    # Encoder input and embedding
    encoder_input = keras.layers.Input(shape=(None,), name="input_1")
    encoder_inp_emb = keras.layers.Embedding(encoder_vocab_size, inp_emb_size, name="embedding_1")(encoder_input)

    # Encoder cell layers
    encoder_seq, *encoder_state = get_cell[cell_type](h_layer_size, activation=cell_activation, return_sequences=True, return_state=True, 
                                                      dropout=dropout, recurrent_dropout=r_dropout, name="encoder_1")(
                                                            encoder_inp_emb
                                                     )
    for i in range(1, n_enc_layers):
        encoder_seq, *encoder_state = get_cell[cell_type](h_layer_size, activation=cell_activation, return_sequences=True, return_state=True, 
                                                          dropout=dropout, recurrent_dropout=r_dropout, name="encoder_"+str(i+1))(
                                                                encoder_seq
                                                         )
    # Decoder input and embedding
    decoder_input = keras.layers.Input(shape=(None,), name="input_2")
    decoder_inp_emb = keras.layers.Embedding(decoder_vocab_size, inp_emb_size, name="embedding_2")(decoder_input)
    decoder_seq = decoder_inp_emb
    # Decoder cell layers
    for i in range(n_dec_layers-1):
        decoder_seq, *_ = get_cell[cell_type](h_layer_size, activation=cell_activation, return_sequences=True, return_state=True, 
                                              dropout=dropout, recurrent_dropout=r_dropout, name="decoder_"+str(i+1))(
                                                    decoder_seq, initial_state=encoder_state
                                             )
    # Decoder last layer
    decoder_seq, *_ = get_cell[cell_type](h_layer_size, activation=cell_activation, return_sequences=True, return_state=True, 
                                          dropout=dropout, recurrent_dropout=r_dropout, name="decoder_1")(
                                                decoder_inp_emb, initial_state=encoder_state
                                         )

    # Attention layer
    attn_out, attn_scores = AttentionLayer(name='attention_1')([encoder_seq, decoder_seq])        # Bahdanau Attention
    # Concat attention output and decoder output
    dense_concat_input = keras.layers.Concatenate(axis=-1, name='concat_layer_1')([decoder_seq, attn_out])

    # Time distributed Softmax FC layer
    decoder_dense_layer = keras.layers.Dense(decoder_vocab_size, activation="softmax", name="dense_1")
    decoder_dense_output = decoder_dense_layer(dense_concat_input)

    # Define the model that will turn encoder_input_data and decoder_input_data into decoder_target_data
    model = keras.Model([encoder_input, decoder_input], decoder_dense_output)

    model.summary(line_length=150)
    return model

### Inference Model (with attention)

In [11]:
def create_attention_inference_model(model):
    
    '''
    Function to return models needed for inference from the original model (without attention).
    Inputs :
        model -- non-attention model used for training
    Outputs :
        encoder_model 
        deocder_model
        n_enc_layers -- number of layers in the encoder(int)
        n_dec_layers -- number of layers in the decoder(int)
    '''
    
    # Calculating number of layers in encoder and decoder
    n_enc_layers, n_dec_layers = 0, 0
    for layer in model.layers:
        n_enc_layers += layer.name.startswith('encoder')
        n_dec_layers += layer.name.startswith('decoder')

    # Encoder input
    encoder_input = model.input[0]      # Input_1
    # Encoder cell final layer
    encoder_cell = model.get_layer("encoder_"+str(n_enc_layers))
    encoder_type = encoder_cell.__class__.__name__
    encoder_seq, *encoder_state = encoder_cell.output
    # Encoder model
    encoder_model = keras.Model(encoder_input, encoder_state)

    # Decoder input
    decoder_input = model.input[1]      # Input_2
    decoder_inp_emb = model.get_layer("embedding_2")(decoder_input)
    decoder_seq = decoder_inp_emb
    # Inputs to decoder layers' initial states
    decoder_states, decoder_state_inputs = [], []
    for i in range(1, n_dec_layers+1):
        if encoder_type == 'LSTM':
            decoder_state_input = [keras.Input(shape=(encoder_state[0].shape[1],), name="input_"+str(2*i+1)), 
                                   keras.Input(shape=(encoder_state[1].shape[1],), name="input_"+str(2*i+2))]
        else:
            decoder_state_input = [keras.Input(shape=(encoder_state[0].shape[1],), name="input_"+str(i+2))]

        decoder_cell = model.get_layer("decoder_"+str(i))
        decoder_seq, *decoder_state = decoder_cell(decoder_seq, initial_state=decoder_state_input)
        decoder_states += decoder_state
        decoder_state_inputs += decoder_state_input

    # Attention layer
    attn_out, attn_scores = model.get_layer('attention_1')([encoder_seq, decoder_seq])        # Bahdanau Attention
    # Concat attention input and decoder output
    dense_concat_input = keras.layers.Concatenate(axis=-1, name='concat_layer_1')([decoder_seq, attn_out])

    # Softmax FC layer
    decoder_dense = model.get_layer("dense_1")
    decoder_dense_output = decoder_dense(dense_concat_input)

    # Decoder model
    decoder_model = keras.Model(
        [encoder_input, decoder_input] + decoder_state_inputs, [attn_scores, decoder_dense_output] + decoder_states
    )

    return encoder_model, decoder_model, n_enc_layers, n_dec_layers

### Beam Decoder

In [12]:
def beam_decoder_infer(model, input_seqs, max_decoder_timesteps, K=1, target_seqs=None, starting_char_enc=0, batch_size=64, attention=True):
    
    
    '''
    Function to do inference on the model using beam decoder.
    Inputs :
        model -- training model
        input_seqs -- input to encoder(numpy array, shape : (None, timesteps))
        max_decoder_timesteps -- Number of timesteps to infer in decoder(int)
        K -- beam width of beam decoder(int, default : 1)
        target_seqs -- expected target(numpy array, shape : (None, timesteps, decoder_vocab_size), deault : None)
                       If None, cross entropy errors won't be calculated.
        starting_char_enc -- Encoding integer for START_CHAR(int, default : 0)
        batch_size -- batch_size sent to Keras predict(int, default : 64)
        attention -- whether the model has attention or not(bool, defualt : False)
        
    Outputs :
        final_outputs -- top K output sequences(numpy array, shape : (None, K, timesteps))
        final_errors -- cross entropy errors for top K output(numpy array, shape : (None, K))
        states_values -- hidden states of decoder(numpy array, shape : (K, None, timesteps, h_layer_size))
        final_attn_scores -- attention to all encoder timesteps for a decoder timestep(numpy array, shape : (None, K, decoder_timesteps, encoder_timesteps))
    '''
    
    # Generating output from encoder
    encoder_model, decoder_model, n_enc_layers, n_dec_layers = create_attention_inference_model(model)
    encoder_output = encoder_model.predict(input_seqs, batch_size=batch_size)
    encoder_out = encoder_output if type(encoder_output) is list else [encoder_output]

    # Number of input samples in the data passed
    no_samples = input_seqs.shape[0]

    # Top K output sequences for each input 
    final_outputs = np.zeros((no_samples, K, max_decoder_timesteps), dtype=np.int32)
    
    # Errors for top K output sequences for each input
    final_errors = np.zeros((no_samples, K))
    
    # Attention scores for top K output sequences for each input
    final_attn_scores = np.zeros((no_samples, K, max_decoder_timesteps, input_seqs.shape[1]))

    # decoder input sequence for 1 timestep (for all samples). Initially one choice only there
    decoder_k_inputs = np.zeros((no_samples, 1, 1))
    
    # Populate the input sequence with the start character at the 1st timestep
    decoder_k_inputs[:, :, 0] = starting_char_enc

    # (log(probability) sequence, decoder output sequence) pairs for all choices and all samples. Probability starts with log(1) = 0
    decoder_k_out = [[(0, [])] for _ in range(no_samples)]
    
    # Categorical cross entropy error in the sequence for all choice and all samples
    errors = [[0] for _ in range(no_samples)]
    
    # Output states from decoder for all choices, and all samples
    states_values  = [encoder_out * n_dec_layers]

    # Attention weights output
    attn_k_scores = [[None] for _ in range(no_samples)]

    # Sampling loop
    for it in range(max_decoder_timesteps):
        # Storing respective data for all possibilities
        All_k_beams = [[] for _ in range(no_samples)]
        All_decoder_states = [[] for _ in range(no_samples)]
        All_errors = [[] for _ in range(no_samples)]
        All_attn_scores = [[] for _ in range(no_samples)]

        for k in range(len(decoder_k_out[0])):
            if attention:
                attn_score, decoder_output, *decoder_states = decoder_model.predict([input_seqs, decoder_k_inputs[:,k]] + states_values[k], batch_size=batch_size)
            else:
                decoder_output, *decoder_states = decoder_model.predict([decoder_k_inputs[:,k]] + states_values[k], batch_size=batch_size)

            # Top K scores
            top_k = np.argsort(decoder_output[:, -1, :], axis=-1)[:, -K:]
            for b in range(no_samples):
                All_k_beams[b] += [(
                    decoder_k_out[b][k][0] + np.log(decoder_output[b, -1, top_k[b][i]]),
                    decoder_k_out[b][k][1] + [top_k[b][i]]
                ) for i in range(K)]

                if attention:
                    All_attn_scores[b] += [attn_score[b]] * K if attn_k_scores[b][k] is None \
                                          else [np.concatenate((attn_k_scores[b][k], attn_score[b]), axis=0)] * K
            
                if target_seqs is not None:
                    All_errors[b] += [errors[b][k] - np.log(decoder_output[b, -1, target_seqs[b, it]])] * K
                
                All_decoder_states[b] += [[state[b:b+1] for state in decoder_states]] * K
        
        # Sort and choose top K with max probabilities
        sorted_ind = list(range(len(All_k_beams[0])))
        sorted_ind = [sorted(sorted_ind, key = lambda ix: All_k_beams[b][ix][0])[-K:][::-1] for b in range(no_samples)]
        
        # Choose the top K decoder output sequences till now
        decoder_k_out = [[All_k_beams[b][ind] for ind in sorted_ind[b]] for b in range(no_samples)]

        # Update the input sequence for next 1 timestep
        decoder_k_inputs = np.array([[All_k_beams[b][ind][1][-1] for ind in sorted_ind[b]] for b in range(no_samples)])

        # Update states
        states_values = [All_decoder_states[0][ind] for ind in sorted_ind[0]]
        for b in range(1, no_samples):
            states_values = [[np.concatenate((states_values[i][j], All_decoder_states[b][ind][j])) 
                              for j in range(len(All_decoder_states[b][ind]))] for i,ind in enumerate(sorted_ind[b])]

        # Update attention scores
        if attention:
            attn_k_scores = [[All_attn_scores[b][ind] for ind in sorted_ind[b]] for b in range(no_samples)]

        # Update errors
        if target_seqs is not None:
            errors = [[All_errors[b][ind] for ind in sorted_ind[b]] for b in range(no_samples)]

    final_outputs = np.array([[decoder_k_out[b][i][1] for i in range(K)] for b in range(no_samples)])
    if target_seqs is not None:
        final_errors = np.array(errors) / max_decoder_timesteps
    if attention:
        final_attn_scores = np.array(attn_k_scores)

    return final_outputs, final_errors, np.array(states_values), final_attn_scores


def calc_metrics(k_outputs, target_seqs, char_enc, char_dec, k_errors=None, exact_word=True):
    
    '''
    Calculates the accuracy (and mean error if info provided) for the best of K possible output sequences
    target_seqs -- Expected output (encoded sequence)
    k_outputs -- k choices of output sequences for each sample
    '''

    matches = np.mean(k_outputs == np.repeat(target_seqs.reshape((target_seqs.shape[0], 1, target_seqs.shape[1])), k_outputs.shape[1], axis=1), axis=-1)
    best_k = np.argmax(matches, axis=-1)
    best_ind = (tuple(range(best_k.shape[0])), tuple(best_k))
    accuracy = np.mean(matches[best_ind])

    loss = None
    if k_errors is not None:
        loss = np.mean(k_errors[best_ind])
    if exact_word:
        equal = [0] * k_outputs.shape[0]
        true_out = convert_to_word(target_seqs, char_enc, char_dec)
        for k in range(k_outputs.shape[1]):
            pred_out = convert_to_word(k_outputs[:,k], char_enc, char_dec)
            equal = [equal[i] or (pred_out[i] == true_out[i]) for i in range(k_outputs.shape[0])]
        exact_accuracy = np.mean(equal)

        return accuracy, exact_accuracy, loss
    
    return accuracy, loss


def beam_decoder(model, input_seqs, target_seqs_onehot, max_decoder_timesteps, char_enc, char_dec, K=1, 
                 model_batch_size=64, attention=True, infer_batch_size=512, exact_word=True, return_outputs=False, 
                 return_states=False, return_attn_scores=False):
    '''
    Function to calculate/capture character-wise accuracy, exact-word-match accuracy, and loss for the seq2seq model using a beam decoder.
    Inputs:
        model -- model used for training
        input_seqs -- input to encoder(numpy array, shape : (None, timesteps))
        target_seqs -- expected target in onehot format(numpy array, shape : (None, timesteps, decoder_vocab_size))
        max_decoder_timesteps -- Number of timesteps to infer in decoder(int)
        char_enc -- target character encoding(dict)
        char_dec -- target character decoding(list)
        K -- beam width to be used in beam decoder(int, default : 1)
        model_batch_size -- batch size to be used while evaluating model using Keras(int, default : 64)
        attention -- whether the model has attention or not(bool, defualt : False)
        infer_batch_size -- number of samples to be sent to beam_decoder_infer() at a time(int, default : 512)
        exact_word -- whether or not exact_accuracy has(bool, default : True)
        return_outputs -- whether or not the outputs predicted need to be returned(bool, default : True)
        return_states -- whether or not the decoder hidden states need to be returned(bool, default : True)
        return_attn_scores -- whether or not the attention scores need to be returned(bool, default : True)
    Outputs:
        accuracy -- the character-wise match accuracy(float)
        (If exact_word is True) exact_accuracy -- (float) the exact word match accuracy
        loss -- (float) the cross-entropy loss for the top K predictions
        (If return_outputs is True) k_outputs -- (numpy ndarray of size : (None, K, timesteps)) top K output sequences
        (If return_states is True) k_states -- (numpy ndarray of size : (K, None, timesteps, h_layer_size))  hidden states of decoder
        (If return_attn_scores is True) k_attn_scores -- (numpy ndarray of size : (None, K, decoder_timesteps, encoder_timesteps)) attention scores
    '''
    
    target_seqs = np.argmax(target_seqs_onehot, axis=-1)
    k_outputs, k_errors, k_states, k_attn_scores = None, None, None, None
    for i in range(0, input_seqs.shape[0], infer_batch_size):
        tmp_k_outputs, tmp_k_errors, tmp_k_states, tmp_k_attn_scores = beam_decoder_infer(model, input_seqs[i:i+infer_batch_size], 
                                                                                          max_decoder_timesteps, K, 
                                                                                          target_seqs[i:i+infer_batch_size], char_enc['\t'], 
                                                                                          model_batch_size, attention)
        if k_errors is None:
            k_outputs, k_errors, k_states, k_attn_scores = tmp_k_outputs, tmp_k_errors, tmp_k_states, tmp_k_attn_scores
        else:
            k_outputs = np.concatenate((k_outputs, tmp_k_outputs))
            k_errors = np.concatenate((k_errors, tmp_k_errors))
            k_states = np.concatenate((k_states, tmp_k_states), axis=2)
            k_attn_scores = np.concatenate((k_attn_scores, tmp_k_attn_scores))

    return_elements = []
    if return_outputs:
        return_elements += [k_outputs]
    if return_states:
        return_elements += [k_states]
    if return_attn_scores:
        return_elements += [k_attn_scores]

    if len(return_elements) > 0:
        return calc_metrics(k_outputs, target_seqs, char_enc, char_dec, k_errors, exact_word) + tuple(return_elements)

    return calc_metrics(k_outputs, target_seqs, char_enc, char_dec, k_errors, exact_word)

### Custom class to calculate word level and charecter level validation accuracy

### Train Function

In [14]:
def train(model, train_input_data, train_target_data, val_input_data, val_target_data, beam_width = 5, attention = True,
                batch_size = 256, optimizer = 'adam', learning_rate = 0.001, epochs = 10, loss_fn = 'categorical_crossentropy'):
    
    # Function to train the model using the mentioned optimizer, learning rate and epochs using given training and validation data

    if optimizer == 'adam':
        model.compile(optimizer = Adam(learning_rate=learning_rate), loss = loss_fn, metrics = ['accuracy'])
    elif optimizer == 'momentum':
        model.compile(optimizer = SGD(learning_rate=learning_rate, momentum = 0.9), loss = loss_fn, metrics = ['accuracy'])
    elif optimizer == 'rmsprop':
        model.compile(optimizer = RMSprop(learning_rate=learning_rate), loss = loss_fn, metrics = ['accuracy'])
    elif optimizer == 'nesterov':
        model.compile(optimizer = SGD(learning_rate=learning_rate, momentum = 0.9, nesterov = True), loss = loss_fn, metrics = ['accuracy'])
    elif optimizer == 'nadam':
        model.compile(optimizer = Nadam(learning_rate=learning_rate), loss = loss_fn, metrics = ['accuracy'])
    else:
        model.compile(optimizer = SGD(learning_rate=learning_rate), loss = loss_fn, metrics = ['accuracy'])

    # Using validation accuracy as the metric
    model.fit(train_input_data,
              train_target_data,
              batch_size = batch_size,
              epochs = epochs
             )

    return model

### Function to calculate levenshtein distance between two sequences
Code refered from: https://codereview.stackexchange.com/questions/217065/calculate-levenshtein-distance-between-two-strings-in-python

In [15]:
def levenshtein_dist(s1, s2):
    # Function to calculate levenshtein distance between two sequences usign Dynamic Programming
    m, n = len(s1)+1, len(s2)+1
    # Initialisation
    dp = np.zeros((m, n))
    # Base case
    dp[0,1:] = np.arange(1,n)
    dp[1:,0] = np.arange(1,m)

    # Recursion
    for i in range(1,m):
        for j in range(1,n):
            if s1[i-1] == s2[j-1]:
                dp[i,j] = min(dp[i-1,j-1], dp[i-1,j]+1, dp[i,j-1]+1)
            else:
                dp[i,j] = min(dp[i,j-1], dp[i-1,j], dp[i-1,j-1]) + 1
    
    return dp[m-1,n-1]

### Test Model

In [16]:
def test(model, test_enc_input, test_dec_target, max_decoder_seq_length, target_char_enc, target_char_dec, test_input=None):
    '''
    Function to evaluate the model metrics on test data and optionally save the predictions.
    Inputs :
        test_enc_input -- input to encoder(numpy array shape : (None, timesteps)) (where characters are encoded as integers)
        test_dec_target -- expected target in onehot format(numpy array shape : (None, timesteps, decoder_vocab_size))
        max_decoder_seq_length -- number of timesteps in the decoder(int)
        target_enc_enc -- target character encoding(dict)
        target_char_dec -- target character decoding(list)
        test_input -- input as words(list of string : (no_samples, input word), default : None) (needed while saving predictions to file alone)
    Outputs :
        acc -- character-wise match accuracy(float)
        exact_K_acc -- exact word match accuracy using the beam width for the model(float)
        exact_acc -- exact word match accuracy using the first prediction (float)(which is equivalent to beam width = 1)
        loss -- loss value(float)
        true_out -- true output(list of string : (no_samples, word))
        pred_out -- predicted output(2D list of string : (no_samples, K, word))
        pred_scores -- levenshtein distance of prediction to true output(2D list : (no_samples, K))
        attn_scores -- attention scores(numpy ndarray of size : (None, K, decoder_timesteps, encoder_timesteps))
        model -- the model obtained from the run
    '''
    

    no_samples, K, batch_size = test_enc_input.shape[0], 5, 64
    acc, exact_K_acc, loss, outputs, attn_scores = beam_decoder(model, test_enc_input, test_dec_target, max_decoder_seq_length, target_char_enc, 
                                                                target_char_dec, K, batch_size, True,
                                                                return_outputs=True, return_attn_scores=True)
    
    print(f'Test accuracy (word level with beam width = {K}) : {exact_K_acc*100:.2f}%')

    test_target = np.argmax(test_dec_target, axis=-1)
    true_out = convert_to_word(test_target, target_char_enc, target_char_dec)
    pred_out = [[] for _ in range(no_samples)]
    pred_scores = [[] for _ in range(no_samples)]
    for k in range(K):
        pred = convert_to_word(outputs[:,k], target_char_enc, target_char_dec)
        pred_out = [pred_out[b] + [pred[b]] for b in range(no_samples)]
        pred_scores = [pred_scores[b] + [levenshtein_dist(pred[b], true_out[b])] for b in range(no_samples)]
    
    equal = [pred_out[i][0] == true_out[i] for i in range(no_samples)]
    exact_acc = np.mean(equal)

    print(f'Test accuracy (word level with the first prediction) : {exact_acc*100:.2f}%')
    print('\n')
    

    # We write the input and top K outputs in decreasing order of probabilities to the file
    pred_file_name = 'predictions_attention.csv'
    with open(pred_file_name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Input"] + ["Prediction_"+str(k+1) for k in range(K)])
        for b in range(no_samples):
            writer.writerow([test_input[b]] + [pred_out[b][k] for k in range(K)])


    return acc, exact_K_acc, exact_acc, loss, true_out, pred_out, pred_scores, attn_scores, model

### Function to print sample prediction

In [17]:
def get_color(value, cmap=None):
    
  # Function to get appropriate color for a value between 0 and 1 from the default blue to red hard-coded colors or a matplotlib cmap 
  
  colors = ['#85c2e1', '#89c4e2', '#95cae5', '#99cce6', '#a1d0e8',
    '#b2d9ec', '#baddee', '#c2e1f0', '#eff7fb', '#f9e8e8',
    '#f9e8e8', '#f9d4d4', '#f9bdbd', '#f8a8a8', '#f68f8f',
    '#f47676', '#f45f5f', '#f34343', '#f33b3b', '#f42e2e']
  if cmap is not None:
      rgba = matplotlib.cm.get_cmap(cmap)(value)
      return 'rgb'+str(tuple([int(c*255) for c in rgba[:-1]]))
  value = min(int((value * 100) / 5), 19)
  return colors[value]

def visualize_samples(input, true_out, pred_out, pred_scores, rand_seq=None):
    
    '''
    Function to print sample outputs in a neat format
    Arguments :
        input -- input words
        true_out -- true output as words
        pred_out -- K predicted output words
        pred_scores -- levenshtein distance for the predictions to the true output
        rand_seq -- list of indices from the dataset passed for which the sample outputs are to be printed (If None, random 10 samples will be chosen)
    Returns :
        rand_seq -- the list of indices for which sample outputs are printed
    '''
    
    n_samples = len(true_out)
    if rand_seq is None:
        rand_seq = np.random.randint(n_samples, size=(10,))
    print('-'*20 + f' Top {len(pred_scores[0])} predictions in decreasing order of probabilities for 10 random samples ' + '-'*20)
    print('')
    for i in rand_seq:
        K = len(pred_scores[i])
        html_str = '''
        <table style="border:2px solid black; border-collapse:collapse">
        <caption> <strong>INPUT :</strong> {} &emsp; | &emsp; <strong> TRUE OUTPUT : </strong> {} </caption>
        <tr>
        <th scope="row" style="border:1px solid black;padding:10px;text-align:left"> Top {} Predictions </th>
        '''.format(input[i], true_out[i], K)
        for k in range(K):
            html_str += '''
            <td style="color:#000;background-color:{};border:1px solid black;padding:10px"> {} </td>
            '''.format(get_color(pred_scores[i][k]/5), pred_out[i][k])
        html_str += '''
        </tr>
        <tr>
        <th scope="row" style="border:1px solid black;padding:10px;text-align:left"> Levenshtein distance (to true output) &emsp; </th>
        '''
        for k in range(K):
            html_str += '''
            <td style="border:1px solid black;padding:10px"> {} </td>
            '''.format(pred_scores[i][k])
        html_str += '''
        </tr>
        </table>
        '''
        p = display(html_print(html_str))
        print('\n\n')
    
    return rand_seq

### Testing with the best model

In [18]:
#Building the model with the best hyperparameters
model = build_model(len(input_char_dec), len(target_char_dec), 
                    inp_emb_size=512, n_enc_layers=2, 
                    n_dec_layers=5, h_layer_size=256, 
                    cell_type='LSTM', dropout=0.3, r_dropout=0.3)


#Training the model with best set of hyperparameters
model = train(model = model, train_input_data= [train_enc_x,train_dec_x], train_target_data= train_dec_y, 
                      val_input_data= [val_enc_x,val_dec_x], val_target_data= val_dec_y, beam_width= 5,
                      attention = True, batch_size= 128, optimizer = 'adam', learning_rate= 0.001, 
                      epochs= 15)
model.save("best_model_attn.h5")

In [19]:
#Testing the model with best set of hyperparameters

test_acc, test_exact_K_acc, test_exact_acc, test_loss, test_true_out,\
test_pred_out, test_pred_scores,attn_scores, model = test(model, test_enc_x, 
                                                          test_dec_y, max_decoder_seq_length, 
                                                          target_char_enc, 
                                                          target_char_dec, test_x)

### Visualizing random samples of predictions

In [20]:
random_samples = visualize_samples(test_x, test_true_out, test_pred_out, test_pred_scores)

### Visualise the best model (with attention)

In [21]:
plot_model(model, to_file="model_attn.png", show_shapes=True)

### Function to Plot attention heatmaps

In [22]:
!pip install wget
import wget
from zipfile import ZipFile

### Downloading Bengali fonts for visualization purpose

In [23]:
filename = 'Hind_Siliguri'
url = 'https://fonts.google.com/download?family=Hind%20Siliguri'

filename_zip = wget.download(url)
filename_zip = filename+'.zip'
with ZipFile(filename_zip, 'r') as z:
    z.printdir()
    print('\Extracting files....')
    z.extractall()
    print('Done')
os.remove(filename_zip)

In [24]:
def plot_attn_heatmaps(input, pred_out, pred_scores, attn_scores, wandb_log=False, rand_seq=None):
    
    '''
    Function to plot attention heatmaps
    Arguments :
        input -- input words
        true_out -- true output as words
        pred_out -- K predicted output words
        pred_scores -- levenshtein distance for the predictions to the true output
        attn_scores -- attention scores
        wandb_log -- whether or not to log the image generated to WANDB(bool, default : False)
        rand_seq -- list of indices from the dataset passed for which the sample outputs are to be printed (If None, random 9 samples will be chosen)
    Returns :
        rand_seq -- the list of indices for which sample outputs are printed
    '''
    
    no_samples = len(pred_out)
    if rand_seq is None:
        rand_seq = np.random.randint(no_samples, size=(9,))
    rand_seq = rand_seq[:9]
    
    plt.close('all')
    fig = plt.figure(figsize=(15,15))
    fig, axes = plt.subplots(3, 3, figsize=(15, 15), constrained_layout=True)
    plt.suptitle('Attention Heatmaps', fontsize='x-large')
    for i,ax in zip(rand_seq, axes.flat):
        K = len(pred_scores[i])
        k = np.argmin(pred_scores[i])
        im = ax.imshow(attn_scores[i,k,:len(pred_out[i][k])+1,:len(input[i])+1].T, vmin=0, vmax=1, cmap='magma')
        ax.set_xticks(range(len(pred_out[i][k])+1))
        ax.set_xticklabels(list(pred_out[i][k])+['<end>'], fontproperties=FontProperties(fname="./HindSiliguri-Regular.ttf"))
        ax.set_yticks(range(len(input[i])+1))
        ax.set_yticklabels(list(input[i])+['<end>'])
        ax.set_ylabel(u'Encoder Input')
        ax.set_xlabel(f'Decoder Output')
        ax.set_title(str(i) + r'$^{th}$ example of Test Set')
        ax.set_aspect("equal")
        ax.grid(False)

    # create colorbar
    fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.7)
    
    # Log in WANDB
    if wandb_log:
        run = wandb.init(project="Seq2SeqLearning", entity="cs21s048-cs21s058", reinit=True)
        wandb.log({'attention_heatmaps': fig})
        run.finish()

    plt.savefig('attention_heatmaps.png')
    plt.show()

    return rand_seq

In [25]:
plot_attn_heatmaps(test_x, test_pred_out, test_pred_scores, attn_scores, False, random_samples)

### Visualizing Attention Connections

In [26]:
def cstr(s, color=None):
  # Function to get text html element
  if color is None:
      return '''<text style="padding:2px; color:#C0C0C0"> {} </text>'''.format(s)
  return '''<text style="color:#000;background-color:{}; padding:2px; color:#FF6699"> {} </text>'''.format(color, s)

In [27]:
def print_connectivity(input, pred_out, pred_scores, attn_scores, dec_char_ind=0):
    '''
    Function to visualize attention for one index of decoder output of one sample
    Inputs :
        input -- sample input word
        pred_out -- K predicted output words for the sample
        pred_scores -- levenshtein distance for the predictions to the true output
        attn_scores -- attention scores
        dec_char_ind -- index of the character in decoder for which the visuzalization is to be done
    Outputs :
        -- None --
    '''
    K = len(pred_scores)
    print('-'*20 + f' Visualizing attention for Top {K} predictions (in decreasing order of probabilities) ' + '-'*20)
    print('')
    html_str = '''
    <table style="border:2px solid black; border-collapse:collapse; font-size:1.5em">
    <caption> <strong>INPUT : </strong> {} </caption>
    <tr>
    <th style="border:1px solid black;padding:10px;text-align:center"> Character in Prediction Focussed </th>
    <th style="border:1px solid black;padding:10px;text-align:center"> Attention Visualization </th>
    </tr>
    '''.format(input)
    for k in range(K):  
        char = pred_out[k][dec_char_ind] if dec_char_ind < len(pred_out[k]) else '&lt end &gt' if dec_char_ind == len(pred_out[k]) else '&lt blank &gt'
        middle_char = pred_out[k][dec_char_ind] if dec_char_ind < len(pred_out[k]) else ''
        end_str = pred_out[k][dec_char_ind+1:] if dec_char_ind < len(pred_out[k])-1 else ''
        html_str += '''
        <tr>
        <td style="border:1px solid black;padding:10px;text-align:center"> character at index {} of {}<span style="color: #FF1493">{}</span>{} <br/> ({}) </td>
        <td style="border:1px solid black;padding:10px;text-align:center">
        '''.format(dec_char_ind, pred_out[k][:dec_char_ind], middle_char, end_str, char)
        for i,c in enumerate(input):
            html_str += '''
            {}
            '''.format(cstr(c, get_color(attn_scores[k,dec_char_ind,i], 'YlGnBu')))
        html_str += '''
        </td>
        </tr>
        '''
    html_str += '''
    </table>
    '''
    display(html_print(html_str))

In [28]:
def visualize_attention(sample_ind=0, dec_char_ind=0):
    
    
    # Function to visualize the importance of encoder input characters to the (dec_char_ind)th character of the output,
    # for the (sample_ind)th sample in the test data
    
    
    print_connectivity(test_x[sample_ind], test_pred_out[sample_ind], test_pred_scores[sample_ind], attn_scores[sample_ind], dec_char_ind)

In [29]:
from __future__ import print_function
import ipywidgets as widgets
from ipywidgets import interact, Layout, IntSlider

'''One can directly use visualize_attention(sample_ind, decoder_index) to get the result if interaction isn't needed, 
but uncommenting below code is easier to use and offers a good way to choose the sample index and decoder index, and seeing the attention for K decoder predictons'''


@interact(sample_ind = IntSlider(min=0, max=len(test_x)-1, step=1, value=10, layout=Layout(width='800px')))
def f(sample_ind):
    print(f'Input : {test_x[sample_ind]}')
    print(f'Top {len(test_pred_out[sample_ind])} predictions : ')
    mx_len = 0
    for pred in test_pred_out[sample_ind]:
        print(pred)
        mx_len = max(mx_len, len(pred))
    
    @interact(character_ind = widgets.IntSlider(min=0, max=mx_len-1, step=1, value=0))
    def g(character_ind):
        visualize_attention(sample_ind, character_ind)